# Remote VS Code
A remote VS code enables us using an IDE inside a HPC cluster. The well know VS code remote extension is used to connect to a server. VS code should automatically uses your master session, if implemented, or just ask you for a TOTP and installs the necessary server side components. However, there is a catch here. VS code will open on the login node which is good for basic things, but if we manage to open it on a compute node we use the computer node we can do a range of things including debugging.

To do this, we implement a thing called proxy jump. I have used proxy jump in past to connect to my PhD desk PC via Sunbird which was publically accessible back in the days. The idea is hop from one node to another via the local network.

The ssh commands allows us to do this. The command is as follows:
```bash
ssh -J user@jump_host user@destination_host
```
Here the `jump_host` is the login node and `destination_host` is the compute node. The `-J` flag is used to specify the jump host.

However, there are better ways. But first we must have access to a computer node. We can get one by submitting an interactive job.

```bash
srun --account=UKAEA-AP002-GPU --partition=ampere --gres=gpu:1 --nodes=1 --time=02:00:00  --pty bash -i
```

Once the job is submitted, we need to know the `hostname` of the compute nodes. This can be done by running the following command:
```bash
(.venv) [ir-shar8@login-q-1 ~]$ squeue -u $USER
             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
           5184031    ampere     bash ir-shar8  R       4:29      1 gpu-q-1
```

It is `gpu-q-1`. To be able to connect to the compute node, we need to add a proxy jump to the `~/.ssh/config` file, so VS Code can add it to the list of hosts in remote connect.

```bash
Host 	VSCODE-CSD3
    	User	 ir-shar8
    	HostName	gpu-q-1
    	ProxyJump	CSD3
    	ControlMaster	auto
    	ControlPath	~/.ssh/control-%C
    	ControlPersist	3600
    	ServerAliveInterval	60
    	ServerAliveCountMax	 5
```

We are making a master connection, so it doesn't ask us TOTP every time. `~/.ssh/control-%C` will not conflict with CSD3's control path because `%C` will be replaced by the host name.

```{note}
We need to change the hostname in the `~/.ssh/config` file to the compute node hostname everytime we submit a new job. 
```

Now we can open VS code and connect to the compute node. Once you open the terminal you are already in the compute node with direct access to the GPU simialr to a  personal PC. If you want to activate a python environment this can be done in `.bashrc` of the HPC or can be done manually.

## Problem jupyter notebooks
Unfortunately, I was not able to run jupyter notebook in the VS code running directly on computer node. I tried a lot of things. If I manage to get it working I will update this doc.  